In [1]:
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind
from scipy.stats import wilcoxon
import pandas as pd
from os.path import join
from itertools import combinations
import numpy as np

In [2]:
csv_folder = '../data/surveys'
csv_files = [join(csv_folder, 'perception_prior.csv'), join(csv_folder, 'perception_after.csv'),
             join(csv_folder, 'perception_diff.csv'), join(csv_folder, 'usability.csv')]

In [25]:
tresh = 0.2

# Perception survey

In [27]:
prior = pd.read_csv(csv_files[0])
after = pd.read_csv(csv_files[1])

In [28]:
prior = prior.sort_values('ID')
after = after.sort_values('ID')

In [29]:
question_codes = list(prior)[4:]
group_key = 'group'

In [30]:
group_list = {}
group_list['group'] = ['Test subject', 'Technicians', 'Lab members', 'Team members']
group_list['group_merged'] = ['Technicians', 'Lab members']

In [31]:
group_results = {}
for g in range(len(group_list[group_key])):
    subset_prior = prior[prior[group_key] == (g+1)].values[:,4:]
    subset_after = after[after[group_key] == (g+1)].values[:,4:]
    group_results[group_list[group_key][g]] = {}
    group_results[group_list[group_key][g]]['prior'] = subset_prior
    group_results[group_list[group_key][g]]['after'] = subset_after

In [32]:
category_results = {}
category_results['ANT'] = {}
category_results['ANT']['prior'] = np.sum(prior.values[:,4:9], axis=1)
category_results['ANT']['after'] = np.sum(after.values[:,4:9], axis=1)

category_results['ANI'] = {}
category_results['ANI']['prior'] = np.sum(prior.values[:,9:15], axis=1)
category_results['ANI']['after'] = np.sum(after.values[:,9:15], axis=1)

category_results['LIKE'] = {}
category_results['LIKE']['prior'] = np.sum(prior.values[:,15:20], axis=1)
category_results['LIKE']['after'] = np.sum(after.values[:,15:20], axis=1)

category_results['INT'] = {}
category_results['INT']['prior'] = np.sum(prior.values[:,20:25], axis=1)
category_results['INT']['after'] = np.sum(after.values[:,20:25], axis=1)

category_results['SAFE'] = {}
category_results['SAFE']['prior'] = np.sum(prior.values[:,25:], axis=1)
category_results['SAFE']['after'] = np.sum(after.values[:,25:], axis=1)

In [33]:
print(wilcoxon(category_results['ANT']['prior'], category_results['ANT']['after']))
print(wilcoxon(category_results['ANI']['prior'], category_results['ANI']['after']))
print(wilcoxon(category_results['LIKE']['prior'], category_results['LIKE']['after']))
print(wilcoxon(category_results['INT']['prior'], category_results['INT']['after']))
print(wilcoxon(category_results['SAFE']['prior'], category_results['SAFE']['after']))

WilcoxonResult(statistic=34.0, pvalue=0.4188677383260413)
WilcoxonResult(statistic=33.0, pvalue=0.03809964229488416)
WilcoxonResult(statistic=63.5, pvalue=0.8121076366682847)
WilcoxonResult(statistic=33.5, pvalue=0.07128589199216011)
WilcoxonResult(statistic=58.0, pvalue=0.9084891707653966)


## All results paired t-test evaluation

In [34]:
prior_all = prior.values[:,4:]
after_all = after.values[:,4:]

In [35]:
t_values, p_values = ttest_rel(prior_all, after_all)

In [36]:
for i, p in enumerate(p_values):
    if p < tresh:
        print("Differences for " + question_codes[i] + " are significant with p=" + str(p) + " index=" + str(i))

Differences for ANT2 are significant with p=0.08262239198015026 index=1
Differences for ANI2 are significant with p=0.009328703651280731 index=6
Differences for ANI3 are significant with p=0.08262239198015021 index=7
Differences for ANI5 are significant with p=0.18571584992925577 index=9
Differences for ANI6 are significant with p=0.06881287894875096 index=10
Differences for INT4 are significant with p=0.023858231279662807 index=19
Differences for INT5 are significant with p=0.16348548685933983 index=20
Differences for SAFE1 are significant with p=0.16624585457006283 index=21
Differences for SAFE2 are significant with p=0.06661962700657047 index=22


## Results per group paired t-test evaluation

In [37]:
ttest_by_groups = {}
for key, value in group_results.items():
    t_values, p_values = ttest_rel(value['prior'], value['after'])
    ttest_by_groups[key] = p_values

In [38]:
for key, values in ttest_by_groups.items():
    print("############ " + key + " ############")
    for i, p in enumerate(values):
        if p < tresh:
            print("Differences for " + question_codes[i] + " are significant with p=" + str(p))

############ Test subject ############
############ Technicians ############
Differences for ANT4 are significant with p=0.02503101581845297
Differences for ANI2 are significant with p=0.10193947882985828
Differences for ANI3 are significant with p=0.10193947882985828
Differences for ANI5 are significant with p=0.10986650103422634
Differences for ANI6 are significant with p=0.17468781426411925
Differences for LIKE3 are significant with p=0.0755868184216124
Differences for LIKE4 are significant with p=0.14123477317626154
Differences for SAFE2 are significant with p=0.1508852728417516
Differences for SAFE3 are significant with p=0.10986650103422634
############ Lab members ############
Differences for ANT2 are significant with p=0.1723082967304001
Differences for ANT5 are significant with p=0.1403463008673793
Differences for ANI4 are significant with p=0.07814074941121574
Differences for ANI5 are significant with p=0.07814074941121574
Differences for LIKE2 are significant with p=0.078140

## Differences between groups

In [39]:
comb = list(combinations(group_list[group_key], 2))
ttest_difference = {}
for c in comb:
    ttest_difference[c[0] + '-' + c[1]] = {}
    for key in ['prior', 'after']:
        t_values, p_values = ttest_ind(group_results[c[0]][key], group_results[c[1]][key])
        ttest_difference[c[0] + '-' + c[1]][key] = p_values

In [40]:
for key, values in ttest_difference.items():
    print("############ " + key + " ############")
    for k, v in values.items():
        print("---------- " + k + " ----------")
        for i, p in enumerate(v):
            if p < tresh:
                print("Differences for " + question_codes[i] + " are significant with p=" + str(p))

############ Test subject-Technicians ############
---------- prior ----------
---------- after ----------
############ Test subject-Lab members ############
---------- prior ----------
---------- after ----------
############ Test subject-Team members ############
---------- prior ----------
---------- after ----------
############ Technicians-Lab members ############
---------- prior ----------
Differences for ANI2 are significant with p=0.15877995255314828
Differences for ANI4 are significant with p=0.1929775830195549
Differences for ANI5 are significant with p=0.1272748556129589
Differences for ANI6 are significant with p=0.15672925707331667
Differences for LIKE1 are significant with p=0.17021572043314817
Differences for LIKE2 are significant with p=0.1818932275010314
Differences for LIKE4 are significant with p=0.0862582346252195
Differences for INT1 are significant with p=0.08608280938036773
Differences for INT2 are significant with p=0.12431288736950233
Differences for SAFE1 are

# Usability survey

In [41]:
usability = pd.read_csv(csv_files[-1])

In [42]:
question_codes = list(usability)[4:]

In [43]:
group_key

'group'

In [44]:
group_results = {}
for g in range(len(group_list[group_key])):
    subset = usability[usability['group'] == (g+1)].values[:,4:]
    group_results[group_list[group_key][g]] = subset

In [45]:
group_results

{'Test subject': array([[3, 4, 2, 5, 2, 4, 4, 3, 2, 3]]),
 'Technicians': array([[1, 5, 3, 4, 4, 3, 5, 3, 3, 3],
        [4, 4, 5, 4, 4, 3, 4, 3, 2, 1],
        [2, 2, 3, 2, 2, 3, 2, 3, 3, 1],
        [5, 2, 4, 3, 3, 3, 4, 4, 4, 2],
        [2, 3, 3, 2, 3, 3, 1, 3, 1, 5],
        [5, 1, 3, 3, 3, 4, 4, 2, 3, 1]]),
 'Lab members': array([[4, 5, 1, 4, 3, 3, 1, 4, 3, 2],
        [2, 4, 1, 5, 3, 2, 2, 3, 2, 4],
        [2, 4, 5, 4, 4, 3, 1, 3, 5, 1],
        [3, 3, 4, 2, 5, 2, 3, 4, 4, 1],
        [1, 5, 2, 4, 4, 2, 1, 3, 3, 4],
        [3, 2, 2, 3, 3, 4, 2, 3, 4, 2],
        [1, 2, 2, 2, 5, 2, 4, 2, 2, 1]]),
 'Team members': array([[3, 3, 4, 4, 3, 1, 2, 2, 4, 3],
        [2, 2, 2, 2, 3, 3, 2, 3, 2, 3],
        [3, 3, 2, 3, 3, 1, 2, 3, 4, 3]])}

## Difference between groups

In [46]:
ttest_difference = {}
for c in comb:
    t_values, p_values = ttest_ind(group_results[c[0]], group_results[c[1]])
    ttest_difference[c[0] + '-' + c[1]] = p_values

In [47]:
for key, values in ttest_difference.items():
    print("############ " + key + " ############")
    for i, p in enumerate(values):
        print("Differences for " + question_codes[i] + " are significant with p=" + str(p))

############ Test subject-Technicians ############
Differences for SUS1 are significant with p=nan
Differences for SUS2 are significant with p=nan
Differences for SUS3 are significant with p=nan
Differences for SUS4 are significant with p=nan
Differences for SUS5 are significant with p=nan
Differences for SUS6 are significant with p=nan
Differences for SUS7 are significant with p=nan
Differences for SUS8 are significant with p=nan
Differences for SUS9 are significant with p=nan
Differences for SUS10 are significant with p=nan
############ Test subject-Lab members ############
Differences for SUS1 are significant with p=nan
Differences for SUS2 are significant with p=nan
Differences for SUS3 are significant with p=nan
Differences for SUS4 are significant with p=nan
Differences for SUS5 are significant with p=nan
Differences for SUS6 are significant with p=nan
Differences for SUS7 are significant with p=nan
Differences for SUS8 are significant with p=nan
Differences for SUS9 are signific